In [1]:
import torch, torchvision
import torchvision.models as models
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image

Deep Residual Learning for Image Recognition

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

https://arxiv.org/abs/1512.03385


Identity Mappings in Deep Residual Networks

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

https://arxiv.org/abs/1603.05027

In [2]:
models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
models.resnet50()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
### Model
# resnet18 = models.resnet18(pretrained=True)
resnet18 = models.resnet18(weights="DEFAULT") # 자동으로 pretrain 된 모델 download

## Dataset
to_tensor = torchvision.transforms.Compose(
                [torchvision.transforms.ToTensor(),
               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])]
                                          )

cifar10 = torchvision.datasets.CIFAR10(root='./', download=True, transform=to_tensor)

dataloader = torch.utils.data.DataLoader(cifar10, batch_size=8, shuffle=True, num_workers=2)

Files already downloaded and verified


In [5]:
for idx, data in enumerate(dataloader):
    
    img, gt = data
    
    print(img.shape)
    
    scores = resnet18(img)
    
    print(scores.shape)
    break

torch.Size([8, 3, 32, 32])
torch.Size([8, 1000])


## Fine-tuning

In [ ]:
## TODO: 1. replace the last FC layer for cifar10
### Hint: 1000 -> 10
model = resnet18(weig)


## TODO: 2. fine tuning the last classifier (FC layer) using the cifar 10 training set.



## TODO: 3. evaluation of the cifar 10 test set.



## Other models

In [14]:
models.resnet50()

models.resnet101()

models.resnet152()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Skip connection

https://pytorch.org/vision/0.8/_modules/torchvision/models/resnet.html


Basic Block vs. Bottleneck Block

![](./data/4_resnet_basicblock_bottleneck.png)

- Bottleneck 구조가 Image Classification에는 좋다고 함
- 다른 Task (Semantic Segmentation, Image 랩슨..?) 복자

# Batch normalization , Layer Normalization,  Instance Normalization, and Group Normalization

![](./data/group_norm.png) 



https://wandb.ai/wandb_fc/GroupNorm/reports/Group-Normalization-in-Pytorch-With-Examples---VmlldzoxMzU0MzMy



https://pytorch.org/docs/stable/generated/torch.nn.GroupNorm.html

- Batch norm : 한 channel에 대하여 normlize가 각각 됨
- Layer normalize는 모든 channel을 한번에 normalize 하지만 하나의 instance에 대해서만 normlize 함
    - Input Size가 정해져있을 때, RNN 같은 구조에서 효율적
- Instance norm은 Batch norm과 똑같지만 하나의 instance에 대해서 진행
    - Batch가 중요하지 않은 경우에 효과적, batch간 correlation이 없는 경우 
- Group norm은 channel 별로 group을 나누어서 진행

In [6]:
import torch, torchvision
import torch.nn as nn
import torchvision.models as models
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image


Batch norm, layer norm, instance norm, group norm

https://pytorch.org/docs/stable/nn.html





https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html#torch.nn.LayerNorm

https://pytorch.org/docs/stable/generated/torch.nn.GroupNorm.html
    

In [2]:
class BatchNorm(nn.Module):
    def __init__(self, in_channel, out_channels):
        super(BatchNorm, self).__init__()
        self.bn = nn.BatchNorm2d(in_channel)
        
    def forward(self,x):
        out = self.bn(x)  #[N, C, HW] -> [N, C, HW]
        
        
        return out

    
## For different sequences, e.g., RNN.
class LayerNorm(nn.Module):
    def __init__(self, in_shape, out_channels):
        super(LayerNorm, self).__init__()
        self.ln = nn.LayerNorm(in_shape, eps=1e-08)

    def forward(self,x):
        out = self.ln(x)  #[N, C, HW] -> [N, C, HW]

        
        return out

    
## For style transfer, domain adaptation.
class InstanceNorm(nn.Module):
    def __init__(self, in_channel, out_channels):
        super(InstanceNorm, self).__init__()
        self.In = nn.InstanceNorm2d(in_channel, eps=1e-08) 

    def forward(self,x):
        out = self.In(x)  #[N, C, HW] -> [N, C, HW]
        return out

    
## stable in small batch size.
class GroupNorm(nn.Module):
    def __init__(self, group_size, in_channel, out_channels):
        super(GroupNorm, self).__init__()
        self.gn = nn.GroupNorm(group_size, in_channel, eps=1e-08)  ## num_group and in_channel

    def forward(self,x):
        out = self.gn(x) #[N, C, HW] -> [N, C, HW]
        
        return out


In [5]:
in_channel = 64
feature = torch.randn(8, in_channel, 120, 120)  ## temp tensor [B, C, H, W]


BN = BatchNorm(in_channel, out_channels=64)

out_feat = BN(feature)

print(out_feat.shape) # shape은 똑같지만 batch 전체가 nomalize 된 값으로 나옴
# 8개가 한번에 normalize 됨

torch.Size([8, 64, 120, 120])


In [8]:
LN = LayerNorm(in_shape=list(feature.shape[1:]), out_channels=64)
# Batch 단위로 실행되는것이 아니기 때문에 shape이 들어가야 함

out_feat = LN(feature)

print(out_feat.shape)  # 8개 각각이 따로 normalize 됨

torch.Size([8, 64, 120, 120])


In [7]:
feature.shape[1:]

torch.Size([64, 120, 120])

In [9]:
IN=InstanceNorm(in_channel, out_channels=64)

out_feat = IN(feature)

print(out_feat.shape) # 8, 64 제외한 120, 120이 한번에 normalizaion 됨

torch.Size([8, 64, 120, 120])


In [10]:
GN=GroupNorm(group_size=2, in_channel=in_channel, out_channels=64)

out_feat = GN(feature)

print(out_feat.shape)  ## 32 / 32, 32개씩 나눠진 channel 들이 normalize 됨

GN=GroupNorm(group_size=4, in_channel=in_channel, out_channels=64)

out_feat = GN(feature)

print(out_feat.shape)  ## 16 / 16 / 16 / 16

torch.Size([8, 64, 120, 120])
torch.Size([8, 64, 120, 120])
